## plot_credit_default.py 代码走读

### 1、导入依赖包

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets.base import get_data_home, Bunch
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from skrules import SkopeRules
from skrules.datasets import load_credit_data

### 2、加载excel数据

In [3]:
data = pd.read_excel('..\data\credit_card.xls', sheet_name='Data', header=1)
dataset = Bunch(data=(data.drop('default payment next month', axis=1)),target=np.array(data['default payment next month']))
X = dataset.data
y = dataset.target
y

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)